In [1]:
import deepSI as dsi
import torch

## Port Hamiltonian Neural Networks in `deepSI`

Model structure given by `pHNN_SUBNET`

$$ \frac{dx}{dt} = \frac{1}{\tau} \left (\left ( J(x) - R(x) \right ) \frac{dH}{dx} + G(x) (u - u_\text{mean})/u_\text{std} \right)$$

$$ (y - y_\text{mean})/y_\text{std} = G(x)^T \frac{dH}{dx}  $$

where 
$$ \tau \text{ is a constant selected as } 10\ T_s  \text{ by default}$$
$$G (\text{Gnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{u}$$
$$J (\text{Jnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{x}\ \text{(skew symetric)}$$
$$R (\text{Rnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{x}\ \text{(semi positive def)}$$
$$H (\text{Hnet}) : n_\text{x} \rightarrow\ \text{scalar}$$
$$u_\text{mean},\ u_\text{std},\ y_\text{mean},\ y_\text{std}\ \text{given by the `norm.umean`, `norm.ustd`, ect.}$$
also `model.integrator(f, x, u, dt)` is a function that integrates the state given a certain state derivative $f$ and input $u$ for $dt$ time. 

These function are constructed by default by using `MLP_res_net` as a base and than converting the output such that it adhers to the constraints. 

In [3]:
norm = dsi.normalization.Norm(0,1,0,1)
na = nb = 3
nx = 4
nu = ny = 'scalar'
model = dsi.models.pHNN_SUBNET(nu, ny, norm, nx, na, nb)

# net(torch.randn(3,4)).shape
r = torch.randn
b = 2
T = 10
upast, ypast, ufuture, yfuture = r(b, nb), r(b, na), r(b, T), r(b, T)
sampling_time = r(b)
model(upast, ypast, ufuture, sampling_time=sampling_time)

tensor([[ 0.6436,  0.6736,  0.6847,  0.6754,  0.6843,  0.6743,  0.6747,  0.6841,
          0.6996,  0.6891],
        [-0.1671, -0.1697, -0.1727, -0.1782, -0.1766, -0.1738, -0.1774, -0.1789,
         -0.1820, -0.1827]], grad_fn=<AddBackward0>)

## Customized function for each element: 

Example (this will get expanded later)

In [ ]:
import deepSI as dsi
import torch
from torch import nn
from deepSI.networks import Quadratic_net

nx = 4
Jnet_bias = dsi.networks.Bias_net(nx*nx)
Jnet_constant = dsi.networks.Contant_net(torch.randn(nx*nx))
Jnet_mlp = dsi.networks.MLP_res_net(input_size=nx, output_size=nx*nx) #simple 
Jnet = dsi.networks.Sum_net([Jnet_bias, Jnet_constant, Jnet_mlp]) #add these three networks together
Jnet = dsi.networks.Skew_sym_converter(Jnet) #x -> nx x nx 
Rnet = dsi.networks.Bias_net(nx*nx)
Rnet = dsi.networks.Sym_pos_semidef_converter(Rnet)

Hnet_depend = dsi.networks.ELU_lower_bound(dsi.networks.MLP_res_net(nx, 'scalar'), lower_bound=-100)
Hnet_qaudratic = Quadratic_net(nx)
Hnet = dsi.networks.Ham_converter(dsi.networks.Sum_net([Hnet_depend,Hnet_qaudratic]))

nu = 'scalar'
ny = 'scalar'

norm = dsi.normalization.Norm(0,1,0,1)
na = nb = 3
model = dsi.models.pHNN_SUBNET(nu, ny, norm, nx, na, nb, Jnet=Jnet, Rnet=Rnet, Hnet=Hnet)

# net(torch.randn(3,4)).shape
r = torch.randn
b = 2
T = 10
upast, ypast, ufuture, yfuture = r(b, nb), r(b, na), r(b, T), r(b, T)
sampling_time = r(b)
model(upast, ypast, ufuture, sampling_time=sampling_time).shape

torch.Size([2, 10])